In [ ]:
import os
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import torchvision.transforms as transforms
from tqdm.notebook import tqdm
import wandb
from transformers import CLIPProcessor, CLIPModel # <--- NEW

# Set Seed
def set_seed(seed=42):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)

set_seed(42)

# Check Device
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"✅ Using device: {DEVICE}")

# Constants
BIDS_ROOT = '/ocean/projects/cis250019p/gandotra/11785-gp-eeg/ds005589'
IMAGE_DIR = '/ocean/projects/cis250019p/gandotra/11785-gp-eeg/images'
CAPTIONS_FILE = '/ocean/projects/cis250019p/gandotra/11785-gp-eeg/captions.txt'
ALL_SUBJECTS = ['sub-02', 'sub-03', 'sub-05', 'sub-09', 'sub-14', 'sub-15', 
                'sub-17', 'sub-19', 'sub-20', 'sub-23', 'sub-24', 'sub-28', 'sub-29']

In [ ]:
class EEG_Paper_Replication_Dataset(Dataset):
    def __init__(self, bids_root, images_dir, captions_path, 
                 subject_list, session_list, 
                 clamp_thres=500, stats=None):
        
        self.bids_root = bids_root
        self.images_dir = images_dir
        self.clamp_thres = clamp_thres
        self.trial_metadata = []
        
        # Subject Mapping
        self.subject_to_idx = {sub: i for i, sub in enumerate(sorted(list(set(subject_list))))}
        
        # 1. Load Captions Helper
        self.captions_dict = self._load_captions(captions_path)
        self.category_to_idx = {cat: i for i, cat in enumerate(sorted(set(c for c, _ in self.captions_dict.values())))}
        
        # 2. Scan Metadata
        print(f"Scanning metadata for {session_list}...")
        for sub in subject_list:
            for ses in session_list:
                for run in ['01', '02', '03', '04']:
                    session_path = os.path.join(self.bids_root, sub, ses)
                    csv_path = os.path.join(session_path, f"{sub}_{ses}_task-lowSpeed_run-{run}_image.csv")
                    npy_path = os.path.join(session_path, f"{sub}_{ses}_task-lowSpeed_run-{run}_1000Hz.npy")
                    
                    if not (os.path.exists(csv_path) and os.path.exists(npy_path)):
                        continue
                        
                    try:
                        csv_data = pd.read_csv(csv_path)
                        for i, row in csv_data.iterrows():
                            img_base_name = self._get_base_name(row['FilePath'])
                            if not img_base_name: continue
                            
                            category, caption = self.captions_dict.get(img_base_name, (None, None))
                            if not category: continue
                            
                            self.trial_metadata.append({
                                'npy_path': npy_path,
                                'trial_index': i,
                                'label': self.category_to_idx[category],
                                'subject_id': self.subject_to_idx[sub],
                                'caption': caption # <--- STORED CAPTION
                            })
                    except:
                        continue
        
        print(f"Found {len(self.trial_metadata)} trials.")

        # 3. Compute Global Statistics
        if stats is None:
            self.mean, self.std = self._compute_global_stats()
        else:
            self.mean, self.std = stats

    def _compute_global_stats(self):
        # (Same logic as before, abbreviated for brevity)
        subset_indices = range(0, len(self.trial_metadata), 10)
        sum_x = 0; sum_sq_x = 0; count = 0
        for i in subset_indices:
            meta = self.trial_metadata[i]
            d = np.load(meta['npy_path'])[meta['trial_index']]
            d = np.clip(d, -self.clamp_thres, self.clamp_thres)
            sum_x += np.mean(d); sum_sq_x += np.mean(d**2); count += 1
        global_mean = sum_x / count
        global_std = np.sqrt((sum_sq_x / count) - (global_mean**2))
        return float(global_mean), float(global_std)

    def get_stats(self): return self.mean, self.std

    def _load_captions(self, path):
        d = {}
        with open(path, 'r') as f:
            next(f)
            for line in f:
                parts = line.strip().split('\t')
                if len(parts) >= 4: d[parts[2]] = (parts[1], parts[3])
        return d

    def _get_base_name(self, p):
        try:
            bn = os.path.splitext(os.path.basename(str(p).replace('\\', '/')))[0]
            if bn.endswith('_resized'): return bn[:-8]
            return bn
        except: return None

    def __len__(self): return len(self.trial_metadata)

    def __getitem__(self, idx):
        meta = self.trial_metadata[idx]
        eeg_data = np.load(meta['npy_path'])[meta['trial_index']]
        eeg_data = np.clip(eeg_data, -self.clamp_thres, self.clamp_thres)
        
        # Standardization
        eeg_data = (eeg_data - self.mean) / (self.std + 1e-6)
        
        return (torch.tensor(eeg_data, dtype=torch.float32), 
                torch.tensor(meta['label'], dtype=torch.long),
                torch.tensor(meta['subject_id'], dtype=torch.long),
                meta['caption']) # <--- RETURN CAPTION STRING

In [ ]:
# 1. Instantiate Training Dataset
print("--- Init Train ---")
train_ds = EEG_Paper_Replication_Dataset(
    bids_root=BIDS_ROOT, 
    images_dir=IMAGE_DIR, 
    captions_path=CAPTIONS_FILE, 
    subject_list=ALL_SUBJECTS, 
    session_list=['ses-01', 'ses-02', 'ses-03'], 
    clamp_thres=500
)
stats = train_ds.get_stats()

# 2. Instantiate Validation Dataset
print("\n--- Init Validation ---")
val_ds = EEG_Paper_Replication_Dataset(
    bids_root=BIDS_ROOT, 
    images_dir=IMAGE_DIR, 
    captions_path=CAPTIONS_FILE, 
    subject_list=ALL_SUBJECTS, 
    session_list=['ses-04'], 
    clamp_thres=500,
    stats=stats
)

train_loader = DataLoader(train_ds, batch_size=32, shuffle=True, num_workers=4, pin_memory=True)
val_loader = DataLoader(val_ds, batch_size=32, shuffle=False, num_workers=4, pin_memory=True)

In [ ]:
class EEG_ViT_1D(nn.Module):
    def __init__(self, num_subjects=13, num_classes=20):
        super().__init__()
        
        # --- Hyperparameters ---
        self.patch_size = 50      # 50ms window 
        self.stride = 25          # 50% overlap
        self.embed_dim = 128      # Feature size
        self.num_heads = 4
        self.depth = 2
        
        # --- 1. Tokenizer (The "Patchify" Step) ---
        # Input: (Batch, 122, 500) -> Output: (Batch, 128, ~19)
        self.tokenizer = nn.Sequential(
            nn.Conv1d(122, self.embed_dim, kernel_size=self.patch_size, stride=self.stride, padding=0),
            nn.BatchNorm1d(self.embed_dim),
            nn.LeakyReLU(0.1)
        )
        
        # --- 2. Learnable "Class Token" ---
        self.cls_token = nn.Parameter(torch.randn(1, 1, self.embed_dim))
        
        # --- 3. Positional Embedding ---
        self.pos_embedding = nn.Parameter(torch.randn(1, 32, self.embed_dim) * 0.01)
        
        # --- 4. Transformer Encoder ---
        encoder_layer = nn.TransformerEncoderLayer(
            d_model=self.embed_dim, 
            nhead=self.num_heads, 
            dim_feedforward=512, 
            dropout=0.5, 
            batch_first=True,
            norm_first=True 
        )
        self.transformer = nn.TransformerEncoder(encoder_layer, num_layers=self.depth)
        
        # --- 5. Subject-Specific Heads ---
        self.heads = nn.ModuleList([
            nn.Linear(self.embed_dim, num_classes) for _ in range(num_subjects)
        ])

    def forward(self, x, subject_ids):
        # Input comes in as (Batch, 500, 122) -> We need (Batch, 122, 500)
        if x.shape[1] == 500 and x.shape[2] == 122:
            x = x.permute(0, 2, 1)
            
        # Safety Crop (in case data is >500)
        if x.shape[2] > 500:
            x = x[:, :, :500]
            
        # ----------------------------------------
        
        # 1. Tokenize
        x = self.tokenizer(x)     # Output: (Batch, 128, 19)
        x = x.permute(0, 2, 1)    # Output: (Batch, 19, 128) -> (Batch, Seq, Dim)
        
        b, seq_len, _ = x.shape
        
        # 2. Append CLS Token
        cls_tokens = self.cls_token.expand(b, -1, -1) 
        x = torch.cat((cls_tokens, x), dim=1) # (Batch, 20, 128)
        
        # 3. Add Positional Embedding
        x = x + self.pos_embedding[:, :seq_len + 1, :]
        
        # 4. Transformer Attention
        x = self.transformer(x)
        
        # 5. Extract ONLY the CLS token output
        cls_output = x[:, 0, :]   # (Batch, 128)
        
        # 6. Subject Routing
        logits = torch.zeros(x.shape[0], 20).to(x.device)
        unique_subs = torch.unique(subject_ids)
        
        for sub in unique_subs:
            mask = (subject_ids == sub)
            logits[mask] = self.heads[sub.long()](cls_output[mask])
            
        return logits

In [ ]:
class EEGClipProbing(nn.Module):
    def __init__(self, original_model, clip_dim=512):
        super().__init__()
        
        # 1. Extract Backbone Components
        self.embed_dim = original_model.embed_dim # Should be 128
        self.tokenizer = original_model.tokenizer
        self.cls_token = original_model.cls_token
        self.pos_embedding = original_model.pos_embedding
        self.transformer = original_model.transformer
        
        # 2. FREEZE BACKBONE
        for param in self.parameters():
            param.requires_grad = False
            
        # 3. Create NEW Heads (Trainable)
        # Maps from Your Model Dim (128) -> CLIP Dim (512)
        self.projection_heads = nn.ModuleList([
            nn.Linear(self.embed_dim, clip_dim) 
            for _ in range(13)
        ])
        
    def forward(self, x, subject_ids):
        # --- FROZEN BACKBONE PASS ---
        with torch.no_grad():
            if x.shape[1] == 500 and x.shape[2] == 122:
                x = x.permute(0, 2, 1)
            if x.shape[2] > 500:
                x = x[:, :, :500]

            x = self.tokenizer(x)     
            x = x.permute(0, 2, 1)    
            
            b, seq_len, _ = x.shape
            cls_tokens = self.cls_token.expand(b, -1, -1) 
            x = torch.cat((cls_tokens, x), dim=1) 
            
            x = x + self.pos_embedding[:, :seq_len + 1, :]
            x = self.transformer(x)
            
            # Extract CLS token
            cls_output = x[:, 0, :]   # (Batch, 128)
            
        # --- TRAINABLE HEAD PASS ---
        output = torch.zeros(b, 512, device=x.device)
        unique_subs = torch.unique(subject_ids)
        
        for sub in unique_subs:
            mask = (subject_ids == sub)
            # This projection head has gradients!
            output[mask] = self.projection_heads[sub.long()](cls_output[mask])
            
        return output

In [ ]:
class ClipAlignmentLoss(nn.Module):
    def __init__(self, temperature=0.07):
        super().__init__()
        self.temperature = temperature
        self.cross_entropy = nn.CrossEntropyLoss()
        
    def forward(self, eeg_emb, text_emb):
        # Normalize vectors
        eeg_emb = F.normalize(eeg_emb, p=2, dim=-1)
        text_emb = F.normalize(text_emb, p=2, dim=-1)
        
        # Similarity Matrix
        logits = (eeg_emb @ text_emb.T) / self.temperature
        
        # Symmetric Loss
        labels = torch.arange(logits.shape[0], device=logits.device)
        loss_e2t = self.cross_entropy(logits, labels)
        loss_t2e = self.cross_entropy(logits.T, labels)
        
        return (loss_e2t + loss_t2e) / 2

In [ ]:
# --- Configuration ---
TASK1_CHECKPOINT = "best_model.pth"       # The file from Task 1 (9.5% Acc)
TASK2_CHECKPOINT = "best_model_task2_frozen.pth" # New file we will save
RESUME_TRAINING = False                          # Set True to continue Task 2 training
TOTAL_EPOCHS = 20                                # Epochs for Linear Probing
LR = 1e-3                                        # Higher LR for heads

# 1. Initialize WandB
wandb.init(
    project="eeg-retrieval",
    name="task2b-linear-probe",
    config={
        "strategy": "Frozen Backbone",
        "architecture": "ViT-1D + Linear Projection",
        "epochs": TOTAL_EPOCHS,
        "lr": LR,
        "batch_size": 32,
        "clip_model": "ViT-B/32"
    }
)

print("\n" + "="*60)
print("SETUP: LINEAR PROBING (BOTH MODELS FROZEN)")
print("="*60)

# 2. Initialize Models
# A. Original Backbone (Structure Only)
original_model = EEG_ViT_1D(num_subjects=13, num_classes=20).to(DEVICE)

# B. Load Task 1 Weights (Pre-requisite)
# We ALWAYS load this first to ensure the backbone is initialized correctly
if os.path.exists(TASK1_CHECKPOINT):
    print(f"✅ Loading Task 1 Backbone from {TASK1_CHECKPOINT}...")
    checkpoint = torch.load(TASK1_CHECKPOINT, map_location=DEVICE)
    if 'model_state_dict' in checkpoint:
        original_model.load_state_dict(checkpoint['model_state_dict'])
    else:
        original_model.load_state_dict(checkpoint)
else:
    print(f"⚠️ WARNING: Task 1 Checkpoint '{TASK1_CHECKPOINT}' not found!")
    print("   The backbone will be random (Training will likely fail).")

# C. Wrap for Task 2 (Freezes backbone, adds new heads)
model = EEGClipProbing(original_model, clip_dim=512).to(DEVICE)

# D. Load CLIP Teacher (Frozen)
print("❄️ Loading & Freezing CLIP Teacher...")
clip_model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32").to(DEVICE)
clip_processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")
for param in clip_model.parameters():
    param.requires_grad = False
clip_model.eval()

# 3. Optimizer & Loss
# Only optimize the new projection heads!
trainable_params = [p for p in model.parameters() if p.requires_grad]
optimizer = optim.AdamW(trainable_params, lr=LR, weight_decay=1e-4)
criterion = ClipAlignmentLoss()

print(f"Trainable Params: {sum(p.numel() for p in trainable_params):,}")

# 4. Resume Logic (Task 2 Specific)
start_epoch = 1
best_val_recall = 0.0

if RESUME_TRAINING and os.path.exists(TASK2_CHECKPOINT):
    print(f"🔄 Resuming Task 2 run from {TASK2_CHECKPOINT}...")
    try:
        checkpoint = torch.load(TASK2_CHECKPOINT, map_location=DEVICE)
        model.load_state_dict(checkpoint['model_state_dict'])
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
        start_epoch = checkpoint['epoch'] + 1
        best_val_recall = checkpoint.get('val_recall', 0.0)
        print(f"   Resuming from Epoch {start_epoch} with Best R@5: {best_val_recall:.2f}%")
    except Exception as e:
        print(f"❌ Error loading Task 2 checkpoint: {e}. Starting fresh.")
else:
    print("🆕 Starting fresh Task 2 training.")

# 5. Training Loop
print(f"\n🚀 Training from Epoch {start_epoch} to {TOTAL_EPOCHS}...")

for epoch in range(start_epoch, TOTAL_EPOCHS + 1):
    model.train()
    total_loss = 0
    
    # --- TRAIN ---
    for batch in tqdm(train_loader, desc=f"Epoch {epoch}", leave=False):
        # Unpack: EEG, Label (unused), SubjectID, Caption
        eeg, _, sub_ids, captions = batch
        eeg, sub_ids = eeg.to(DEVICE), sub_ids.to(DEVICE)
        
        # A. Get CLIP Targets (No Grad)
        with torch.no_grad():
            text_inputs = clip_processor(
                text=list(captions), 
                padding=True, truncation=True, return_tensors="pt"
            ).to(DEVICE)
            text_features = clip_model.get_text_features(**text_inputs)
            
        # B. Get Student Predictions (Grad)
        eeg_features = model(eeg, sub_ids)
        
        # C. Update
        loss = criterion(eeg_features, text_features)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
        
    avg_loss = total_loss / len(train_loader)
    
    # --- VALIDATION (Recall@K) ---
    model.eval()
    all_eeg = []
    all_text = []
    
    with torch.no_grad():
        for batch in val_loader:
            eeg, _, sub_ids, captions = batch
            eeg, sub_ids = eeg.to(DEVICE), sub_ids.to(DEVICE)
            
            # Student
            e_emb = model(eeg, sub_ids)
            e_emb = F.normalize(e_emb, p=2, dim=-1)
            all_eeg.append(e_emb.cpu())
            
            # Teacher
            t_inputs = clip_processor(
                text=list(captions), 
                padding=True, truncation=True, return_tensors="pt"
            ).to(DEVICE)
            t_features = clip_model.get_text_features(**t_inputs)
            t_features = F.normalize(t_features, p=2, dim=-1)
            all_text.append(t_features.cpu())
            
    # Compute Metrics
    eeg_tensor = torch.cat(all_eeg)
    text_tensor = torch.cat(all_text)
    
    # Similarity & Ranking
    sims = eeg_tensor @ text_tensor.T
    n = len(sims)
    targets = torch.arange(n)
    
    # Top-K Accuracy
    top5_preds = sims.topk(5, dim=1).indices
    r1 = (top5_preds[:, 0] == targets).float().mean().item() * 100
    r5 = (top5_preds == targets.unsqueeze(1)).any(dim=1).float().mean().item() * 100
    
    print(f"Epoch {epoch:02d}: Loss {avg_loss:.4f} | Val R@1 {r1:.2f}% | Val R@5 {r5:.2f}%")
    
    # Log to WandB
    wandb.log({
        "epoch": epoch,
        "train_loss": avg_loss,
        "val_r1": r1,
        "val_r5": r5
    })
    
    # Save Checkpoint
    if r5 > best_val_recall:
        best_val_recall = r5
        
        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'val_recall': r5,
            'train_loss': avg_loss
        }, TASK2_CHECKPOINT)
        
        wandb.save(TASK2_CHECKPOINT)
        print(f"  ✅ Best Model Saved! ({r5:.2f}%)")

wandb.finish()